# OGGM glacier simulator

App to initialize a simple glacier and look how it develops under changing mass balance profiles.

## Import packages

import plotting libaries

In [ ]:
import holoviews as hv
from holoviews import opts
from holoviews.streams import Stream, param
import panel as pn

# is needed for linecolor by velocity/thickness colorplots
from bokeh.transform import linear_cmap

hv.extension('bokeh', width=100)
pn.extension()

import OGGM packages

In [ ]:
# Constants for initialization
from functools import partial
from oggm.core.flowline import FluxBasedModel, RectangularBedFlowline, TrapezoidalBedFlowline, ParabolicBedFlowline
from oggm.core.massbalance import LinearMassBalance
from oggm import cfg
cfg.initialize()

# There are several solvers in OGGM core. We use the default one for this experiment
FlowlineModel = partial(FluxBasedModel, min_dt=0, cfl_number=0.01)

and some other useful packages

In [ ]:
import numpy as np

## Define model domain

In [ ]:
# number of steps from bottem to top of glacier
nx = 200

# model grid spacing in m
map_dx = 100

# distance along glacier (x-axis of glacier profil plot) in km
distance_along_glacier = np.linspace(0, nx, nx) * map_dx * 1e-3

# glacier top height
glacier_top_height = 4000

# glacier bottom height
glacier_bottom_height = 0

## Define default values for parameters and range for menu sliders

In [ ]:
# ELA height
ELA_height_min = 0
ELA_height_max = 4000
ELA_height_default = 3000
ELA_height_step = 100

# mass balance gradient
mb_gradient_max = 15
mb_gradient_min = 1
mb_gradient_default = 4
mb_gradient_step = 1

# years of calculation
years_min = 10
years_max = 1000
years_default = 150
years_step = 10

# linear bed rock slope
bed_rock_slope_values = ['39°', '22°', '15°', '11°', '7°', '3°']
bed_rock_slope_default = '11°'

# bed rock profile
bed_rock_profile_values = {'en': ['linear', 'getting flatter',
                                  'getting steeper', 'cliff'],
                           'de': ['linear', 'flächer werdend',
                                  'steiler werdend', 'Klippe']}
bed_rock_profile_default = {'en': 'linear',
                            'de': 'linear'}

# bed rock width
bed_rock_width_values = {'en': ['constant', 'getting narrower', 'getting wider',
                                'wide top, narrow bottom'],
                         'de': ['konstant', 'enger werden', 'breiter werden',
                                'oben weit, unten eng']}
bed_rock_width_default = {'en': 'constant',
                          'de': 'konstant'}

# Glen's creep parameter
glens_creep_parameter_values = {'en': ['small', 'medium', 'large'],
                                'de': ['klein', 'mittel', 'groß']}
glens_creep_parameter_default = {'en': 'medium',
                                 'de': 'mittel'}

# sliding parameter
sliding_parameter_values = {'en': ['no sliding', 'sliding'],
                            'de': ['kein Gleiten', 'Gleiten']}
sliding_parameter_default = {'en': 'no sliding',
                             'de': 'kein Gleiten'}

# model dyears (how often the model state should be shown)
dyears_model_start = 1
dyears_model_end = 100
dyears_model_step = 1
dyears_model_default = 10

# maximum calculation years of model (before it gets interrupted)
max_calc_years_start = 100
max_calc_years_end = 5000
max_calc_years_step = 100
max_calc_years_default = 1000

## Define global model variables:

In [ ]:
bed_h = np.linspace(glacier_top_height, 0, nx)
mb_model = []
model = []
run_surface_heights = []
glacier_outlines = []
glen_a = cfg.PARAMS['glen_a']
fs = 0

# define width, so glacier bed could be created when plot is created (after creation model is ready)
widths = np.zeros(nx) + 4.

# years after the model progress should be shown
dyears = dyears_model_default

# years after model should be aborted
max_calc_years = max_calc_years_default

## Define some plotting variables

In [ ]:
# default tools for plots
default_tools_geometry = ['save', 'box_zoom', 'reset']
default_tools_timeseries = ['pan', 'save', 'box_zoom',
                            'xwheel_zoom', 'hover', 'reset']
# is needed so xwheel_zoom is activated by default
default_active_tools_timeseries = ['xwheel_zoom']

# Variable to follow which text is currently displayed in the headerline
header_text_status = 0

# colormap for velocity and thickness
colormap = ['#5ebaff', '#00faf4', '#ffffcc',
            '#ffe775', '#ffc140', '#ff8f20', '#ff6060']

# define nan array for glacier outline plotting in width plot
nan_array = np.empty(nx)
nan_array.fill(np.nan)

# index to force a 'change' for geometry figure
geometry_figure_change_index = True

# define variables for tracking change in glacier volume and length
glacier_volume = [0]
glacier_area = [0]
glacier_length = [0]

# define variable for tracking time corresponding to glacier volume and length
glacier_time = [0]

# index to force a 'change' for timeseries figure (dynamic volume and length plot)
timeseries_figure_change_index = True

# define values for table
years_table = []
ELA_table = []
mb_gradient_table = []
glen_a_table = []
fs_table = []
volume_table = []
length_table = []
area_table = []

# index to force a 'change' for timeseries dynamic map
timeseries_table_change_index = True

# variable to store vertical lines position in volume and length plot
timeseries_vlin_pos = [0]

# for the ELA position in the width plot
ELA_x_position = 0

# variables to calculate glacier wide mass balance
glacier_wide_mb_area = []
glacier_wide_mb_heights = []
glacier_wide_mb_height_mb = []
glacier_wide_mb = []

# define default language
language = 'en'

# index to force a 'language change' for ColorBar
dynamic_ColorBar_change_index = True

# is needed when language is changed
last_info_text_stage = 'new model'

## Define global curves (elements of subplots)

In [ ]:
bed_rock_height_curve = []
bed_rock_width_curve = []
glacier_height_curve = []
mb_curve = []
width_curve = []
info_text = []
glacier_volume_curve = []
glacier_area_curve = []
glacier_length_curve = []
timeseries_table = []

## Define Headerline with help functionality

Define appearance of Headerline and also implement the help functionality, so the help text is displayed in the Headerline

### define actual help function with help text

In [ ]:
def help_function():
    global header_text_status

    # to switch next and privous button clickability on/off at start and end of help
    if header_text_status == 0:
        previous_help_button.disabled = True
        next_help_button.disabled = False
    elif header_text_status == 12:
        previous_help_button.disabled = False
        next_help_button.disabled = True
    else:
        previous_help_button.disabled = False
        next_help_button.disabled = False

    # change helptext and activate desired tab in menu
    if header_text_status == 0:
        header_text.object = {'en': ('<div style="font-size:20px">OGGM edu glacier simulator</div>' +
                                     '<p style="margin-top: 0px;">' +
                                     'With this app you can simulate idealized glaciers using the ' +
                                     '<a href="https://oggm.org">OGGM</a> model. If you are not ' +
                                     'familiar with the app click on "next" to go through the ' +
                                     'instruction or go to the "find help here" section to ' +
                                     'jump to a certain chapter of the help. <br>This app was realised ' +
                                     'using tools of <a href="https://holoviz.org">HoloViz</a>.' +
                                     '</p>'),
                              'de': ('<div style="font-size:20px">OGGM edu Gletschersimulator</div>' +
                                     '<p style="margin-top: 0px;">' +
                                     'Mit dieser App können idealisierte Gletscher mithilfe des ' +
                                     '<a href="https://oggm.org">OGGM</a> Models simuliert werden. ' +
                                     'Wenn du das erstemal hier bist kannst du auf "weiter" drücken ' +
                                     'und dir wird eine Beschreibung in der Kopfzeile angezeigt. Um ' +
                                     'zwischen den Kapiteln der Hilfe zu springen schau unter ' +
                                     '"hier Hilfe finden".<br>Diese App wurde mit tools von ' +
                                     '<a href="https://holoviz.org">HoloViz</a> realisiert.' +
                                     '</p>')
                              }[language]

    elif header_text_status == 1:
        header_text.object = {'en': ('<div style="font-size:18px">beginner mode</div>' +
                                     '<p style="margin-top: 0px;">' +
                                     'In the beginner mode you can change four parameters of the model: ' +
                                     'the slope and the widthprofile of the glacierbed, the ELA ' +
                                     '(equilibrium line altitude) and the gradient of the mass balance profile.' +
                                     'The changes are becoming visible in the figures after the ' +
                                     '"run the model" button is clicked. In the beginner mode the ' +
                                     'model will always start from zero and will run until it reached ' +
                                     'equilibrium (volume stayed constant between two succeeding timesteps).' +
                                     '</p>'),
                              'de': ('<div style="font-size:18px">Anfänger</div>' +
                                     '<p style="margin-top: 0px;">' +
                                     'Hier können vier Parameter des Models verändert werden: die Hangneigung, ' +
                                     'die Gletscherweite, die ELA (Gleichgewichtslinie) und der Gradient des ' +
                                     'Massenbilanzprofils. Die Änderungen werden in den Diagrammen sichtbar ' +
                                     'nachdem der "Model starten" Knopf gedrückt wird. Das Model startet ' +
                                     'jedesmal von Null und läuft solange ' +
                                     'bis ein Gleichgewicht erreicht wird (Volumen bleibt konstant zwischen ' +
                                     'zwei aufeinanderfolgenden Zeitschritten).' +
                                     '</p>')
                              }[language]

        menu.active = 0

    elif header_text_status == 2:
        header_text.object = {'en': ('<div style="font-size:18px">advanced mode (I of II)</div>' +
                                     '<p style="margin-top: 0px;">' +
                                     'First lets look at the ' +
                                     '"run the model" tab. Here you can change the ELA (equilibrium line ' +
                                     'altitude), the gradient of the mass balance profile, Glen\'s creep ' +
                                     'parameter, if there should be sliding at the glacierbed and the last ' +
                                     'slider defines for how long the model should be running with the chosen ' +
                                     'settings. In the advanced mode the model always continue from the current ' +
                                     'glacier state. By clicking "advance model" the model will run for the ' +
                                     'chosen years or with "run to equilibrium" the model runs until it ' +
                                     'reaches equilibrium with the current settings (ignoring the last slider).'
                                     '</p>'),
                              'de': ('<div style="font-size:18px">Fortgeschritten (I von II)</div>' +
                                     '<p style="margin-top: 0px;">' +
                                     'Als erstes zum "Modellauf" tab. Hier können die ELA (Gleichgewichtslinie), ' +
                                     'der Gradient des Massenbilanzprofils, der Glen\'s creep Parameter, das ' +
                                     'Gleiten am Gletscherbed (Ja/Nein) und die länge des nächsten Modellaufs ' +
                                     'eingestellt werden. Hier startet der Modellauf immer vom aktuellen ' +
                                     'Modelstand. Mit "Model starten" läuft das Model für die ausgewählte ' +
                                     'Dauer und mit "bis zum Gleichgewicht" läuft das Model bis es ' +
                                     'im Gleichgewicht (konstantes Volumen) mit den gewählten Einstellungen ist.' +
                                     '</p>')
                              }[language]

        menu.active = 1
        advanced_panel.active = 0

    elif header_text_status == 3:
        header_text.object = {'en': ('<div style="font-size:18px">advanced mode (II of II)</div>' +
                                     '<p style="margin-top: 0px;">' +
                                     'Now looking at the "create new model" tab. Here you can start with a ' +
                                     'new model by defining the bedrock profile, if the linear profile is ' +
                                     'selected also the slope can be defined, and the width profile. By ' +
                                     'clicking the "create new model" button the changes are getting visible ' +
                                     'in the figures and a new model with zero extend is initialised.'
                                     '</p>'),
                              'de': ('<div style="font-size:18px">Fortgeschritten (II von II)</div>' +
                                     '<p style="margin-top: 0px;">' +
                                     'Jetzt zum "neues Model" tab. Hier kann ein neues Model von Null ' +
                                     'gestartet werden. Dafür kann das Untergrundprofil, die ' +
                                     'Gletscherweite und die Hangneigung (wenn das Untergrundprofil ' +
                                     'linear gewählt wurde) gewählt werden. Mit einem Klick auf ' +
                                     '"neues Model" werden die Änderungen wirksam und im ' +
                                     'Diagramm angezeigt.'
                                     '</p>')
                              }[language]

        menu.active = 1
        advanced_panel.active = 1

    elif header_text_status == 4:
        header_text.object = {'en': ('<div style="font-size:18px">color options</div>' +
                                     '<p style="margin-top: 0px;">' +
                                     'Here you can switch additional colors in the geometry plot on and off. ' +
                                     'In the top left figure the ice velocity and in the bottom left figure ' +
                                     'the ice thickness can be shown. Both use the same colorbar which is also ' +
                                     'shown here. The colorbar rearrange the upper limit according to the ' +
                                     'current maximum values (shown in info text of geometry plot). ' +
                                     'The colors can be switched on after the model run or before it. ' +
                                     'But when the colors are switched on during the model run the calculation ' +
                                     'takes much longer.'
                                     '</p>'),
                              'de': ('<div style="font-size:18px">Farboptionen</div>' +
                                     '<p style="margin-top: 0px;">' +
                                     'Hier können zusätzlich Farbschatierungen in der "Geometrie Darstellung" ' +
                                     'an- bzw. ausgeschaltet werden. Links oben kann die Eisgeschwindigkeit ' +
                                     'und links unten die Eisdicke angezeigt werden. Beide Werte verwenden ' +
                                     'die selbe Farbskala, welche auch hier gezeigt wird. Die Farbsakala ' +
                                     'ist dabei dynamisch, das heißt das Maximum der Farbskala bezieht sich ' +
                                     'auf die jeweiligen Maximalwerte welche im info text (rechts unten) ' +
                                     'gezeigt werden. Die Farben können vor oder nach einem Modellauf ' +
                                     'angezeigt werden. Modelläufe mit eingeschaltenen Farben benötigen ' +
                                     'allerdings eine längere Rechenzeit.' +
                                     '</p>')
                              }[language]

        menu.active = 2

    elif header_text_status == 5:
        header_text.object = {'en': ('<div style="font-size:18px">model options</div>' +
                                     '<p style="margin-top: 0px;">' +
                                     'In this section you can define how often the current model state should be ' +
                                     'shown in the plots ("show model progress every ... years") and you can ' +
                                     'define a upper limit of calculation years when the model runs to equilibrium ' +
                                     '("maximum calculation year of model"). The changes came into operation after ' +
                                     'clicking the "set new model options" button. The current settings are shown ' +
                                     'in the text (bold numbers).'
                                     '</p>'),
                              'de': ('<div style="font-size:18px">Modeloptionen</div>' +
                                     '<p style="margin-top: 0px;">' +
                                     'Hier kann definiert werden wieoft der Modelstatus während dem Modellauf ' +
                                     'angezeigt werden soll ("zeige Modelfortschritt alle ... Jahre") und ' +
                                     'es kann eine obere Grenze für einen Modellauf festgelegt werden ' +
                                     '("Maximale Laufzeit eines Modellaufs (Jahre)"). Die Änderungen werden ' +
                                     'mit einem Klick auf "neue Optionen festlegen" wirksam. Die aktuellen ' +
                                     'Einstellungen werden im Text gezeigt (fettgeschriebene Zahlen).' +
                                     '</p>')
                              }[language]

        menu.active = 3

    elif header_text_status == 6:
        header_text.object = {'en': ('<div style="font-size:18px">choose plots</div>' +
                                     '<p style="margin-top: 0px;">' +
                                     'Here you can decide which plots you want to use/should be updated. ' +
                                     'So if you only are interested in one of the plots you can save ' +
                                     'calculation time when only select the desired one. If one plot is ' +
                                     'not selected this will be indicated on the plot.'
                                     '</p>'),
                              'de': ('<div style="font-size:18px">Darstellung wählen</div>' +
                                     '<p style="margin-top: 0px;">' +
                                     'Hier kann ausgewählt werden welche Darstellung benutzt/aktualisiert ' +
                                     'werden soll. So kann Rechenzeit eingespart werden wenn man nur an ' +
                                     'einer Darstellung interessiert ist. Wenn eine Darstellung nicht ' +
                                     'aktiviert ist wird das in der Darstellung selbst angezeigt.' +
                                     '</p>')
                              }[language]

        menu.active = 4

    elif header_text_status == 7:
        header_text.object = {'en': ('<div style="font-size:18px">find help here</div>' +
                                     '<p style="margin-top: 0px;">' +
                                     'With the buttons you can jump directly to one of the help pages you want ' +
                                     'to know more about. (Also to this page here ;).)'
                                     '</p>'),
                              'de': ('<div style="font-size:18px">hier Hilfe finden</div>' +
                                     '<p style="margin-top: 0px;">' +
                                     'Mit den Knöpfen gelangt man direkt zu einzelnen Kapiteln der Hilfe. ' +
                                     '(Auch zu dieser Seite hier ;).)'
                                     '</p>')
                              }[language]

        menu.active = 5

    elif header_text_status == 8:
        header_text.object = {'en': ('<div style="font-size:18px">geometry plot (I of III)</div>' +
                                     '<p style="margin-top: 0px;">' +
                                     'The top left figure shows the glacierbed from the side in gray. ' +
                                     'Underneath on the left the figure shows the glacier from above ' +
                                     'and the dark gray areas indicate the boarder of the glacierbed.' +
                                     'In both figures the dark blue line indicates the current glacier ' +
                                     'state and if using the advanced mode also the previous state is ' +
                                     'shown with a dashed line.' +
                                     '</p>'),
                              'de': ('<div style="font-size:18px">Geometrie Darstellung (I von III)</div>' +
                                     '<p style="margin-top: 0px;">' +
                                     'Das Bild oben links zeigt das Gletscherbettprofil von der Seite in grau. ' +
                                     'Darunter ist das Getscherbett von oben gezeigt (Gletscherweite), die ' +
                                     'dunkelgrauen Flächen bilden die Grenze. In beiden Bildern wird der aktuelle ' +
                                     'Gletscherstand als weiße Fläche mit blauem Rand dargestellt. Wenn man das ' +
                                     'Model mit "Fortgeschritten" betreibt, wird auch der letzte Gletscherstand ' +
                                     '(Start des aktuellen Modellaufes) mit einer strichlieren Linie angezeigt.'
                                     '</p>')
                              }[language]

        figures.active = 0

    elif header_text_status == 9:
        header_text.object = {'en': ('<div style="font-size:18px">geometry plot (II of III)</div>' +
                                     '<p style="margin-top: 0px;">' +
                                     'The top right figure shows the height dependent annual mass ' +
                                     'balance as a black solid line and the dashed black line shows the' +
                                     'equilibrium line altitude (ELA, which is also shown in the figures ' +
                                     'on the left). The green area indicates altitudes with a positive ' +
                                     'mass balance (mass gain) and the red area shows altitudes with a ' +
                                     'negative mass balance (mass loss) for the glacier. The text ' +
                                     'underneath shows statistics of the current glacier (model) state.' +
                                     '</p>'),
                              'de': ('<div style="font-size:18px">Geometrie Darstellung (II von III)</div>' +
                                     '<p style="margin-top: 0px;">' +
                                     'Das Bild rechts oben zeigt die höhenabhängige jährliche Massenbilanz ' +
                                     'als schwarze Linie. Die strichlierte Linie zeigt die Höhe der ELA an (auch in ' +
                                     'den anderen zwei Bildern zu sehen). Grüne Flächen zeigen einen Massengewinn am ' +
                                     'Gletscher an und rote Flächen einen Massenverlust. Der Text unten rechts zeigt ' +
                                     'aktuelle Werte des Models an.'
                                     '</p>')
                              }[language]

        figures.active = 0

    elif header_text_status == 10:
        header_text.object = {'en': ('<div style="font-size:18px">geometry plot (III of III)</div>' +
                                     '<p style="margin-top: 0px;">' +
                                     'If you want to zoom in you can do this by clicking into one of ' +
                                     'the figures, hold the click and draw an rectangle. All figures ' +
                                     'will rezoom automatically to the same extent. To go back to the ' +
                                     'original extent click on the "Reset" button to the right of the ' +
                                     'figures (two arrows forming a circle).' +
                                     '</p>'),
                              'de': ('<div style="font-size:18px">Geometrie Darstellung (III von III)</div>' +
                                     '<p style="margin-top: 0px;">' +
                                     'Wenn du hineinzoomen möchtest kannst du mit der Maus ein Rechteck ' +
                                     'aufziehen (linke Maustaste gedrückt halten). Alle Bilder zoomen ' +
                                     'automatisch auf den gleichen Ausschnitt. Um den Zoom rückgängig ' +
                                     'zu machen auf "Reset" rechts neben den Bildern drücken (zwei ' +
                                     'Pfeile welche einen Kreis formen).' +
                                     '</p>')
                              }[language]

        figures.active = 0

    elif header_text_status == 11:
        header_text.object = {'en': ('<div style="font-size:18px">timeseries plot (I of II)</div>' +
                                     '<p style="margin-top: 0px;">' +
                                     'The three figures on the left show the evolution of the length, area ' +
                                     'and length with time. The resolution of the figures (how ' +
                                     'often the state is shown) depends on the model settings (see ' +
                                     '"model options"). After the model run has finished the section ' +
                                     'gets a number and some model settings and statistics are shown ' +
                                     'in the table on the right. The figures and the table are cleared ' +
                                     'when a new model is initialized.' +
                                     '</p>'),
                              'de': ('<div style="font-size:18px">Zeitreihen Darstellung (I von II)</div>' +
                                     '<p style="margin-top: 0px;">' +
                                     'Die drei Bilder rechts zeigen die zeitlichen Verläufe der Länge, Fläche und ' +
                                     'Volumen. Die Auflösung hängt dabei von den aktuellen Modeleinstellungen ab ' +
                                     '(siehe "Modeloptionen"). Wenn ein Modellauf beendet wurde bekommt der ' +
                                     'Abschnitt eine Nummer und es werden Modeleinstellungen und Endwerte ' +
                                     'des Models in der Tabelle rechts angezeigt. Die Bilder und die Tabelle ' +
                                     'werden gelöscht wenn ein neues Model gestartet wird.'
                                     '</p>')
                              }[language]

        figures.active = 1

    elif header_text_status == 12:
        header_text.object = {'en': ('<div style="font-size:18px">timeseries plot (II of II)</div>' +
                                     '<p style="margin-top: 0px;">' +
                                     'You can zoom in and out on the plots by using the mousewheel ' +
                                     'and you can use a mouseclick to drag the plots around. If you ' +
                                     'zoom in on one plot the other plot will automatically rezoom ' +
                                     'to the same time span. When moving with the mouse over the curves ' +
                                     'a hover will show you the exact values of the points. To go back ' +
                                     'to the original extent click on the "Reset" button to the left of ' +
                                     'the figures (two arrows forming a circle).'
                                     '</p>'),
                              'de': ('<div style="font-size:18px">Zeitreihen Darstellung (II von II)</div>' +
                                     '<p style="margin-top: 0px;">' +
                                     'Mit dem Mausrad kann in die Bilder hineingezoomt werden und mit einem ' +
                                     'Mausklick kann der Bildausschnitt verschoben werden. Alle Änderungen ' +
                                     'des Bildausschnitts werden automatisch auf den anderen Bildern ' +
                                     'aktualisiert. Mit "Reset" (zwei Pfeile welche einen Kreis formen) ' +
                                     'links von den Bildern kann der Zoom rückgängig gemacht werden. ' +
                                     'Wenn man mit dem Mauszeiger über die Kurven fährt ' +
                                     'werden die genauen Werte der Zeitschritte angezeigt.'
                                     '</p>')
                              }[language]

        figures.active = 1

### help functions for Headerline widgets

In [ ]:
def next_help_button_click(arg=None):
    global header_text_status

    header_text_status += 1

    help_function()


def previous_help_button_click(arg=None):
    global header_text_status

    header_text_status -= 1

    help_function()

In [ ]:
def language_selecter_function(arg=None):
    global language
    language = language_selecter.value
    change_language()


def change_language(arg=None):
    global geometry_figure_change_index
    global timeseries_figure_change_index
    global timeseries_table_change_index

    # set language of headerline
    set_headerline_language()

    # set language of menu widgets
    set_beginner_panel_language()
    set_advanced_panel_widget_language()
    set_advanced_panel_tabs()
    set_help_panel_language()
    set_color_option_panel_language()
    set_model_option_panel_language()
    set_choose_plots_panel_language()

    # set language of menu titles
    set_menu_tabs()

    # set tab-title language of figuers tab menu
    set_figures_tabs()

    # set language of geometry plot
    set_bed_rock_curves()
    set_glacier_height_curve()
    set_width_curve()
    set_mb_curve()
    set_info_text(stage=last_info_text_stage)
    geometry_figure.event(change=geometry_figure_change_index)
    geometry_figure_change_index = not geometry_figure_change_index

    # set language of timeseries plot
    set_glacier_length_curve()
    set_glacier_area_curve()
    set_glacier_volume_curve()
    set_timeseries_table()
    timeseries_figure.event(change=timeseries_figure_change_index)
    timeseries_figure_change_index = not timeseries_figure_change_index
    timeseries_table_DynamicMap.event(change=timeseries_table_change_index)
    timeseries_table_change_index = not timeseries_table_change_index

### define elements of Headerline

function to set Headerline language

In [ ]:
def set_headerline_language():
    next_help_button.name = {'en': 'next',
                             'de': 'weiter'
                             }[language]

    previous_help_button.name = {'en': 'previous',
                                 'de': 'zurück'
                                 }[language]

    # changes language of help text
    help_function()

define elements

In [ ]:
header_text = pn.pane.Markdown(sizing_mode='stretch_width', height=100)

next_help_button = pn.widgets.Button(disabled=False,
                                     button_type='success',
                                     width=75)
next_help_button.param.watch(next_help_button_click, 'clicks')

previous_help_button = pn.widgets.Button(disabled=True,
                                         button_type='danger',
                                         width=75)
previous_help_button.param.watch(previous_help_button_click, 'clicks')

language_selecter = pn.widgets.RadioBoxGroup(name='select your language',
                                             options=['en', 'de'],
                                             inline=True,
                                             margin=(0, 0),
                                             width=70)
language_selecter.param.watch(language_selecter_function, 'value')

oggm_edu_logo = '<a href="http://edu.oggm.org"><img src="http://edu.oggm.org/en/latest/_images/oggm.gif" width=170></a>'
hv_logo = '<a href="https://holoviz.org"><img src="https://holoviz.org/assets/holoviz-logo-stacked.svg" width=80></a>'

set_headerline_language()

### Put actual Headerline together

In [ ]:
header = pn.Row(pn.Pane(oggm_edu_logo),
                pn.layout.Spacer(width=10),
                header_text,
                pn.Column(pn.layout.Spacer(),
                          previous_help_button,
                          next_help_button),
                pn.layout.Spacer(),
                pn.Column(pn.layout.Spacer(),
                          language_selecter,
                          pn.Pane(hv_logo, width=60)),
                sizing_mode='stretch_width')

## Define Panels for tab menu

Defining the individual menus and putting them together in an tab menu.

In [ ]:
# define background color of tab menu
menu_background = '#fafafa'

### Panel for beginner mode

beginner button function

In [ ]:
def beginner_button_click(arg=None):
    # set Variables not changeable in beginner mode to there default values
    bed_rock_profile.value = bed_rock_profile_default[language]
    glens_creep_parameter.value = glens_creep_parameter_default[language]
    sliding_parameter.value = sliding_parameter_default[language]

    # for consitency set values of the advanced mode the same as choosen in beginner mode
    mb_gradient_advanced.value = mb_gradient.value
    ELA_height_advanced.value = ELA_height.value
    bed_rock_width_advanced.value = bed_rock_width.value
    bed_rock_slope_advanced.value = bed_rock_slope.value

    # set numeric values out of selection
    set_glen_a()
    set_fs()

    run_the_model(stage='beginner mode')

function so change language

In [ ]:
def set_beginner_panel_language():
    bed_rock_slope.name = {'en': 'slope of bedrock profile',
                           'de': 'Hangneigung'
                           }[language]

    bed_rock_width.name = {'en': 'width along glacier',
                           'de': 'Gletscherweite'
                           }[language]

    mb_gradient.name = {'en': 'mass balance gradient',
                        'de': 'Massenbilanzgradient'
                        }[language]

    ELA_height.name = {'en': 'ELA (equilibrium line altitude)',
                       'de': 'ELA (Gleichgewichtslinie)'
                       }[language]

    beginner_button.name = {'en': 'run the model',
                            'de': 'Model starten'
                            }[language]

    bed_rock_width.options = bed_rock_width_values[language]

actual beginner Panel

In [ ]:
bed_rock_slope = pn.widgets.Select(options=bed_rock_slope_values,
                                   value=bed_rock_slope_default,
                                   sizing_mode='stretch_width')

bed_rock_width = pn.widgets.Select(value=bed_rock_width_default[language],
                                   sizing_mode='stretch_width')

mb_gradient = pn.widgets.IntSlider(start=mb_gradient_min,
                                   end=mb_gradient_max,
                                   step=mb_gradient_step,
                                   value=mb_gradient_default,
                                   sizing_mode='stretch_width')

ELA_height = pn.widgets.IntSlider(start=ELA_height_min,
                                  end=ELA_height_max,
                                  step=ELA_height_step,
                                  value=ELA_height_default,
                                  sizing_mode='stretch_width')

beginner_button = pn.widgets.Button(button_type='primary',
                                    margin=(15, 0, 0, 0),
                                    sizing_mode='stretch_width')
beginner_button.param.watch(beginner_button_click, 'clicks')

set_beginner_panel_language()

beginner_panel = pn.Row(bed_rock_slope,
                        bed_rock_width,
                        mb_gradient,
                        ELA_height,
                        beginner_button,
                        background=menu_background,
                        sizing_mode='stretch_width')

### Panel for advanced mode

advanced button help functions

In [ ]:
def advance_model_click():
    # for consitency set values of the beginner mode the same as choosen in advanced mode
    mb_gradient.value = mb_gradient_advanced.value
    ELA_height.value = ELA_height_advanced.value
    bed_rock_width.value = bed_rock_width_advanced.value
    bed_rock_slope.value = bed_rock_slope_advanced.value

    # set numeric values out of selection
    set_glen_a()
    set_fs()

advanced button functions

In [ ]:
def run_model_equilibrium_button_click(arg=None):
    advance_model_click()
    run_the_model(stage='advance equilibrium')


def run_model_years_button_click(arg=None):
    advance_model_click()
    run_the_model(stage='advance years')


def create_new_model_button_click(arg=None):
    # set sliders of beginner mode to the same as choosen in advanced mode
    bed_rock_width.value = bed_rock_width_advanced.value
    bed_rock_slope.value = bed_rock_slope_advanced.value

    init_model()

help function for selecting slope when profile is linear

In [ ]:
def bed_rock_profile_change(arg=None):
    if bed_rock_profile.value == 'linear':
        bed_rock_slope_advanced.disabled = False
    else:
        bed_rock_slope_advanced.disabled = True

function to change the language

In [ ]:
def set_advanced_panel_widget_language():
    mb_gradient_advanced.name = {'en': 'mass balance gradient',
                                 'de': 'Massenbilanzgradient'
                                 }[language]

    bed_rock_profile.name = {'en': 'bedrock profile',
                             'de': 'Untergrundprofil'
                             }[language]

    ELA_height_advanced.name = {'en': 'ELA',
                                'de': 'ELA'
                                }[language]

    bed_rock_slope_advanced.name = {'en': 'slope of linear bedrock profile',
                                    'de': 'Hangneigung (lineares Untergrundprofil)'
                                    }[language]

    bed_rock_width_advanced.name = {'en': 'width along glacier',
                                    'de': 'Gletscherweite'
                                    }[language]

    glens_creep_parameter.name = {'en': "Glen's creep parameter",
                                  'de': "Glen's creep Parameter"
                                  }[language]

    sliding_parameter.name = {'en': 'Sliding parameter',
                              'de': 'gleiten'
                              }[language]

    run_model_years.name = {'en': 'years to advance model',
                            'de': 'Jahre für Modellauf'
                            }[language]

    run_model_equilibrium_button.name = {'en': 'run to equilibrium',
                                         'de': 'bis zum Gleichgewicht'
                                         }[language]

    run_model_years_button.name = {'en': 'advance model',
                                   'de': 'Model starten'
                                   }[language]

    create_new_model_button.name = {'en': 'create new model',
                                    'de': 'neues Model'
                                    }[language]

    bed_rock_profile.options = bed_rock_profile_values[language]
    bed_rock_profile.value = bed_rock_profile_default[language]

    bed_rock_width_advanced.options = bed_rock_width_values[language]
    bed_rock_width_advanced.value = bed_rock_width_default[language]

    glens_creep_parameter.options = glens_creep_parameter_values[language]
    glens_creep_parameter.value = glens_creep_parameter_default[language]

    sliding_parameter.options = sliding_parameter_values[language]
    sliding_parameter.value = sliding_parameter_default[language]

actual advanced Panel

In [ ]:
mb_gradient_advanced = pn.widgets.IntSlider(start=mb_gradient_min,
                                            end=mb_gradient_max,
                                            step=mb_gradient_step,
                                            value=mb_gradient_default,
                                            sizing_mode='stretch_width')

bed_rock_profile = pn.widgets.Select(sizing_mode='stretch_width')
bed_rock_profile.param.watch(bed_rock_profile_change, 'value')

ELA_height_advanced = pn.widgets.IntSlider(start=ELA_height_min,
                                           end=ELA_height_max,
                                           step=ELA_height_step,
                                           value=ELA_height_default,
                                           sizing_mode='stretch_width')

bed_rock_slope_advanced = pn.widgets.Select(options=bed_rock_slope_values,
                                            value=bed_rock_slope_default,
                                            sizing_mode='stretch_width')

bed_rock_width_advanced = pn.widgets.Select(sizing_mode='stretch_width')

glens_creep_parameter = pn.widgets.Select(sizing_mode='stretch_width')

sliding_parameter = pn.widgets.RadioBoxGroup(margin=(10, 0, 0, 0),
                                             sizing_mode='stretch_width')

run_model_years = pn.widgets.IntSlider(start=years_min,
                                       end=years_max,
                                       step=years_step,
                                       value=years_default,
                                       margin=(5, 0, 0, 0),
                                       sizing_mode='stretch_width')

run_model_equilibrium_button = pn.widgets.Button(button_type='primary',
                                                 sizing_mode='stretch_width')
run_model_equilibrium_button.param.watch(
    run_model_equilibrium_button_click, 'clicks')

run_model_years_button = pn.widgets.Button(button_type='primary',
                                           sizing_mode='stretch_width')
run_model_years_button.param.watch(run_model_years_button_click, 'clicks')

create_new_model_button = pn.widgets.Button(button_type='primary',
                                            sizing_mode='stretch_width',
                                            margin=(15, 0, 0, 0))
create_new_model_button.param.watch(create_new_model_button_click, 'clicks')

set_advanced_panel_widget_language()

advanced_panel = pn.Tabs(('', []),
                         ('', []),
                         background=menu_background,
                         tabs_location='left')

advanced_panel_first_tab = pn.Row(ELA_height_advanced,
                                  mb_gradient_advanced,
                                  glens_creep_parameter,
                                  pn.Row(sliding_parameter,
                                         width=90),
                                  run_model_years,
                                  pn.Column(run_model_years_button,
                                            run_model_equilibrium_button,
                                            sizing_mode='stretch_width'),
                                  sizing_mode='stretch_width')

advanced_panel_second_tab = pn.Row(bed_rock_profile,
                                   bed_rock_slope_advanced,
                                   bed_rock_width_advanced,
                                   create_new_model_button,
                                   sizing_mode='stretch_width')


# is necessary to change the tabs title language
def set_advanced_panel_tabs():
    advanced_panel.__setitem__(0, ({'en': 'run the model',
                                    'de': 'Modellauf'
                                    }[language],
                                   advanced_panel_first_tab))

    advanced_panel.__setitem__(1, ({'en': 'create new model',
                                    'de': 'neues Model'
                                    }[language],
                                   advanced_panel_second_tab))


set_advanced_panel_tabs()

### Panel for help

help functions for buttons

In [ ]:
def beginner_mode_help_button_click(arg=None):
    global header_text_status

    # choose the start page
    header_text_status = 1
    help_function()


def advanced_mode_help_button_click(arg=None):
    global header_text_status

    # choose the start page
    header_text_status = 2
    help_function()


def color_options_help_button_click(arg=None):
    global header_text_status

    # choose the start page
    header_text_status = 4
    help_function()


def model_options_help_button_click(arg=None):
    global header_text_status

    # choose the start page
    header_text_status = 5
    help_function()


def choose_plots_help_button_click(arg=None):
    global header_text_status

    # choose the start page
    header_text_status = 6
    help_function()


def find_help_here_help_button_click(arg=None):
    global header_text_status

    # choose the start page
    header_text_status = 7
    help_function()


def geometry_plot_help_button_click(arg=None):
    global header_text_status

    # choose the start page
    header_text_status = 8
    help_function()


def timeseries_plot_help_button_click(arg=None):
    global header_text_status

    # choose the start page
    header_text_status = 11
    help_function()

function to change the language

In [ ]:
def set_help_panel_language():
    help_intro_text.object = {'en': ('<p style="margin-top: 0px;">Click on button to get help shown in ' +
                                     'Headerline. Navigate with "next" and "previous" button (on the right).' +
                                     '</p>'),
                              'de': ('<p style="margin-top: 0px;">Auf Button drücken um Hilfe in der ' +
                                     'Kopfzeile anzuzeigen. Navigieren mit dem "weiter" und ' +
                                     '"zurück" Button (Kopfzeile rechts)')
                              }[language]

    beginner_mode_help_button.name = {'en': 'beginner mode',
                                      'de': 'Anfänger'
                                      }[language]

    advanced_mode_help_button.name = {'en': 'advanced mode',
                                      'de': 'Fortgeschritten'
                                      }[language]

    color_options_help_button.name = {'en': 'color options',
                                      'de': 'Farboptionen'
                                      }[language]

    model_options_help_button.name = {'en': 'model options',
                                      'de': 'Modeloptionen'
                                      }[language]

    choose_plots_help_button.name = {'en': 'choose plots',
                                     'de': 'Darstellung wählen'
                                     }[language]

    find_help_here_help_button.name = {'en': 'find help here',
                                       'de': 'hier Hilfe finden'
                                       }[language]

    geometry_plot_help_button.name = {'en': 'geometry plot',
                                      'de': 'Geometrie Darstellung'
                                      }[language]

    timeseries_plot_help_button.name = {'en': 'timeseries plot',
                                        'de': 'Zeitreihen Darstellung'
                                        }[language]

actual help Panel

In [ ]:
help_intro_text = pn.pane.Markdown(sizing_mode='stretch_width', height=20)

beginner_mode_help_button = pn.widgets.Button(button_type='primary',
                                              sizing_mode='stretch_width')
beginner_mode_help_button.param.watch(
    beginner_mode_help_button_click, 'clicks')

advanced_mode_help_button = pn.widgets.Button(button_type='primary',
                                              sizing_mode='stretch_width')
advanced_mode_help_button.param.watch(
    advanced_mode_help_button_click, 'clicks')

color_options_help_button = pn.widgets.Button(button_type='primary',
                                              sizing_mode='stretch_width')
color_options_help_button.param.watch(
    color_options_help_button_click, 'clicks')

model_options_help_button = pn.widgets.Button(button_type='primary',
                                              sizing_mode='stretch_width')
model_options_help_button.param.watch(
    model_options_help_button_click, 'clicks')

choose_plots_help_button = pn.widgets.Button(button_type='primary',
                                             sizing_mode='stretch_width')
choose_plots_help_button.param.watch(
    choose_plots_help_button_click, 'clicks')

find_help_here_help_button = pn.widgets.Button(button_type='primary',
                                               sizing_mode='stretch_width')
find_help_here_help_button.param.watch(
    find_help_here_help_button_click, 'clicks')

geometry_plot_help_button = pn.widgets.Button(button_type='primary',
                                              sizing_mode='stretch_width')
geometry_plot_help_button.param.watch(
    geometry_plot_help_button_click, 'clicks')

timeseries_plot_help_button = pn.widgets.Button(button_type='primary',
                                                sizing_mode='stretch_width')
timeseries_plot_help_button.param.watch(
    timeseries_plot_help_button_click, 'clicks')

set_help_panel_language()

help_panel = pn.Row(help_intro_text,
                    pn.Column(beginner_mode_help_button,
                              advanced_mode_help_button,
                              sizing_mode='stretch_width'),
                    pn.Column(color_options_help_button,
                              model_options_help_button,
                              sizing_mode='stretch_width'),
                    pn.Column(choose_plots_help_button,
                              find_help_here_help_button,
                              sizing_mode='stretch_width'),
                    pn.Column(geometry_plot_help_button,
                              timeseries_plot_help_button,
                              sizing_mode='stretch_width'),
                    background=menu_background,
                    sizing_mode='stretch_width')

### Panel for color options

help functions

In [ ]:
def show_velocity_changed(arg=None):
    global geometry_figure_change_index
    set_glacier_height_curve()
    geometry_figure.event(change=geometry_figure_change_index)
    geometry_figure_change_index = not geometry_figure_change_index


def show_thickness_changed(arg=None):
    global geometry_figure_change_index
    set_width_curve()
    geometry_figure.event(change=geometry_figure_change_index)
    geometry_figure_change_index = not geometry_figure_change_index

function to change the language

In [ ]:
def change_ColorBar_language(change=True):
    return ColorBar.opts(colorbar_opts={'orientation': 'horizontal',
                                        'major_label_overrides': {-1: '0',
                                                                  -0.5: {'en': 'quater',
                                                                         'de': 'viertel'
                                                                         }[language],
                                                                  0: {'en': 'half',
                                                                      'de': 'hälfte'
                                                                      }[language],
                                                                  0.5: {'en': 'three-quaters',
                                                                        'de': 'dreiviertel'
                                                                        }[language],
                                                                  1: {'en': 'maximum',
                                                                      'de': 'Maximum'
                                                                      }[language]
                                                                  },
                                        'major_label_text_font_size': '10pt'})

In [ ]:
def set_color_option_panel_language():
    global dynamic_ColorBar_change_index

    show_velocity.name = {'en': 'show velocity as colors (top left figure)',
                          'de': 'Eisgeschwindigkeit als Farbe (oben links)'
                          }[language]

    show_thickness.name = {'en': 'show thickness as colors (bottom left figure)',
                           'de': 'Eisdicke als Farbe (unten links)'
                           }[language]

    color_options.object = {'en': ('<p style="margin-top: 0px;">Here you can choose if the velocity or/and the thickness ' +
                                   'of the ice should be shown. The colorbar shows the intervals, ' +
                                   'maximum refers to the value shown in the "info text".</p> '),
                            'de': ('<p style="margin-top: 0px;">Hier kann ausgewählt werden ob die Eisgeschwindigkeit ' +
                                   'und/oder die Eisdicke als Farbe angezeigt werden soll. Die gezeigte Farbskala ' +
                                   'bezieht sich auf die maximal Werte im "info Text" (unten rechts). </p>')
                            }[language]

    dynamic_ColorBar.event(change=dynamic_ColorBar_change_index)
    dynamic_ColorBar_change_index = not dynamic_ColorBar_change_index

actual panel

In [ ]:
show_velocity = pn.widgets.Checkbox(sizing_mode='stretch_width',
                                    margin=(2, 0, 0, 0))
show_velocity.param.watch(show_velocity_changed, 'value')

show_thickness = pn.widgets.Checkbox(sizing_mode='stretch_width',
                                     margin=(0, 0))
show_thickness.param.watch(show_thickness_changed, 'value')

color_options = pn.pane.Markdown(height=41,
                                 sizing_mode='stretch_width',
                                 margin=0)


# is needed to directly get to bokeh styling options
def colorbar_hook(plot, element):
    plot.handles['plot'].border_fill_color = menu_background
    plot.handles['plot'].min_border_right = 50
    plot.handles['plot'].min_border_bottom = 0
    plot.handles['colorbar'].background_fill_color = menu_background
    plot.handles['colorbar'].height = 7


ColorBar = hv.Polygons({'x': [],
                        'y': [],
                        'value': [0]},
                       vdims='value').opts(hooks=[colorbar_hook],
                                           responsive=True,
                                           colorbar_position='top',
                                           yaxis=None,
                                           xaxis=None,
                                           # show_frame=False,
                                           toolbar=None,
                                           frame_height=0,
                                           cmap=colormap,
                                           colorbar=True)

ColorBar_stream = Stream.define('ColorBar_stream', change=False)

dynamic_ColorBar = hv.DynamicMap(
    change_ColorBar_language, streams=[ColorBar_stream()])

set_color_option_panel_language()

color_option_panel = pn.Column(pn.Row(color_options,
                                      pn.Column(show_velocity,
                                                show_thickness,
                                                background=menu_background,
                                                sizing_mode='stretch_width')),
                               pn.Row(dynamic_ColorBar,
                                      height=40,
                                      sizing_mode='stretch_width'),
                               background=menu_background,
                               sizing_mode='stretch_width')

### Panel for model options

help function for button click

In [ ]:
def set_model_options(arg=None):
    global dyears
    global max_calc_years

    dyears = dyears_model_slider.value
    max_calc_years = max_calc_years_slider.value
    current_model_options.object = current_model_options_text.format(
        dyears_model_slider.value, max_calc_years_slider.value)

function to change the language

In [ ]:
def set_model_option_panel_language():
    global current_model_options_text

    dyears_model_slider.name = {'en': 'show model progress every ... years',
                                'de': 'zeige Modelfortschritt alle ... Jahre'
                                }[language]

    max_calc_years_slider.name = {'en': 'maximum calculation year of model',
                                  'de': 'Maximale Laufzeit eines Modellaufs (Jahre)'
                                  }[language]

    current_model_options_text = {'en': ('Model currently showing progress every **{}** years and ' +
                                         'abort model after **{}** years of calculation'),
                                  'de': ('Model zeigt Fortschritt alle **{}** Jahre und Modellauf ' +
                                         'wird nach **{}** Jahren abgebrochen')
                                  }[language]
    current_model_options.object = current_model_options_text.format(
        dyears_model_slider.value, max_calc_years_slider.value)

    set_model_options_button.name = {'en': 'set new model options',
                                     'de': 'neue Optionen festlegen'
                                     }[language]

actual panel

In [ ]:
dyears_model_slider = pn.widgets.IntSlider(start=dyears_model_start,
                                           end=dyears_model_end,
                                           step=dyears_model_step,
                                           value=dyears_model_default,
                                           sizing_mode='stretch_width')

max_calc_years_slider = pn.widgets.IntSlider(start=max_calc_years_start,
                                             end=max_calc_years_end,
                                             step=max_calc_years_step,
                                             value=max_calc_years_default,
                                             sizing_mode='stretch_width')

current_model_options = pn.pane.Markdown(sizing_mode='stretch_width')
# set global so it can be changed according to language
current_model_options_text = []

set_model_options_button = pn.widgets.Button(button_type='primary',
                                             sizing_mode='stretch_width')
set_model_options_button.param.watch(set_model_options, 'clicks')

set_model_option_panel_language()

model_option_panel = pn.Row(current_model_options,
                            dyears_model_slider,
                            max_calc_years_slider,
                            set_model_options_button,
                            background=menu_background,
                            sizing_mode='stretch_width')

### Panel to choose used figures

help function for button click

In [ ]:
# functions to set an infotext in the plots telling if they
# are active or not


def switch_geometry_plot(arg=None):
    global geometry_figure_change_index
    if show_geometry_plot.value:
        set_info_text(stage='switched on')
    else:
        set_info_text(stage='switched off')
    geometry_figure.event(change=geometry_figure_change_index)
    geometry_figure_change_index = not geometry_figure_change_index


def switch_timeseries_plot(arg=None):
    global timeseries_figure_change_index
    global timeseries_table_change_index

    init_timeseries_plot()

    timeseries_figure.event(change=timeseries_figure_change_index)
    timeseries_figure_change_index = not timeseries_figure_change_index
    timeseries_table.event(change=timeseries_table_change_index)
    timeseries_table_change_index = not timeseries_table_change_index

function to change the language

In [ ]:
def set_choose_plots_panel_language():
    show_geometry_plot.name = {'en': 'show geometry plot',
                               'de': 'zeige Geometrie Darstellung'
                               }[language]

    show_timeseries_plot.name = {'en': 'show timeseries plot',
                                 'de': 'zeige Zeitreihen Darstellung'
                                 }[language]

actual panel

In [ ]:
show_geometry_plot = pn.widgets.Checkbox(value=True)
show_geometry_plot.param.watch(switch_geometry_plot, 'value')

show_timeseries_plot = pn.widgets.Checkbox(value=False)
show_timeseries_plot.param.watch(switch_timeseries_plot, 'value')

set_choose_plots_panel_language()

choose_plots_panel = pn.Column(show_geometry_plot,
                               show_timeseries_plot,
                               background=menu_background,
                               sizing_mode='stretch_width')

### add all panels together as tabs

In [ ]:
menu = pn.Tabs(('', []),
               ('', []),
               ('', []),
               ('', []),
               ('', []),
               ('', []),
               sizing_mode='stretch_width',
               background=menu_background)


# is necassary to change the tabs title language
def set_menu_tabs():
    menu.__setitem__(0, ({'en': 'beginner mode',
                          'de': 'Anfänger'
                          }[language],
                         beginner_panel))

    menu.__setitem__(1, ({'en': 'advanced mode',
                          'de': 'Fortgeschritten'
                          }[language],
                         advanced_panel))

    menu.__setitem__(2, ({'en': 'color options',
                          'de': 'Farboptionen'
                          }[language],
                         color_option_panel))

    menu.__setitem__(3, ({'en': 'model options',
                          'de': 'Modeloptionen'
                          }[language],
                         model_option_panel))

    menu.__setitem__(4, ({'en': 'choose plots',
                          'de': 'Darstellung wählen'
                          }[language],
                         choose_plots_panel))

    menu.__setitem__(5, ({'en': 'find help here',
                          'de': 'hier Hilfe finden'
                          }[language],
                         help_panel))


set_menu_tabs()

## Define functions to set model parameters

Functions to set model parameters, which are needed to initialize a new model

### set bed h

In [ ]:
def set_bed_h():
    global bed_h

    # look which bed rock profile was chosen
    if bed_rock_profile.value in ['linear', 'linear']:
        # look which slope was choosen and set the heigth at each point along the glacier
        if bed_rock_slope.value == '39°':
            bed_h = np.append(np.linspace(glacier_top_height, 0, np.round(nx / 4)),
                              np.zeros(nx - np.int(np.round(nx / 4))))
        elif bed_rock_slope.value == '22°':
            bed_h = np.append(np.linspace(glacier_top_height, 0, np.round(nx / 4) * 2),
                              np.zeros(nx - np.int(np.round(nx / 4) * 2)))
        elif bed_rock_slope.value == '15°':
            bed_h = np.append(np.linspace(glacier_top_height, 0, np.round(nx / 4) * 3),
                              np.zeros(nx - np.int(np.round(nx / 4) * 3)))
        elif bed_rock_slope.value == '11°':
            bed_h = np.linspace(glacier_top_height, 0, nx)
        elif bed_rock_slope.value == '7°':
            bed_h = np.linspace(glacier_top_height, 1500, nx)
        elif bed_rock_slope.value == '3°':
            bed_h = np.linspace(glacier_top_height, 3000, nx)

    elif bed_rock_profile.value in ['getting flatter', 'flächer werdend']:
        bed_h = np.array([(np.log(x / nx + 0.03) / np.log(0.5) * 0.2 + 0.03) *
                          glacier_top_height for x in np.arange(nx)])

    elif bed_rock_profile.value in ['getting steeper', 'steiler werdend']:
        bed_h = np.array([(np.log(-x / nx / 4 + 0.26) * 0.2 / np.log(2) + 1.3) *
                          glacier_top_height for x in np.arange(nx)])

    elif bed_rock_profile.value in ['cliff', 'Klippe']:
        # define extend of cliff
        cliff_top = 2500
        cliff_bottom = 2400

        bed_h = np.concatenate((np.linspace(glacier_top_height, cliff_top, nx/2),
                                np.linspace(cliff_bottom, glacier_bottom_height, nx/2)))

### set widths

In [ ]:
def set_widths():
    global widths

    # look which glacier width was selected
    if bed_rock_width.value in ['constant', 'konstant']:
        widths = np.zeros(nx) + 4.

    elif bed_rock_width.value in ['getting wider', 'breiter werden']:
        widths = np.array([(2 / nx * x + 0.5) * 2 for x in np.arange(nx)])

    elif bed_rock_width.value in ['getting narrower', 'enger werden']:
        widths = np.array([(-2 / nx * x + 2.5) * 2 for x in np.arange(nx)])

    elif bed_rock_width.value in ['wide top, narrow bottom', 'oben weit, unten eng']:
        widths = np.append(np.ones(np.int(np.round(nx/5))) * 4,
                           np.ones(np.int(nx - np.round(nx/5))) * 2)

    # set length in m
    widths = widths * map_dx

### set glen a

In [ ]:
def set_glen_a():
    global glen_a

    if glens_creep_parameter.value in ['medium', 'mittel']:
        glen_a = cfg.PARAMS['glen_a']
    elif glens_creep_parameter.value in ['small', 'klein']:
        glen_a = cfg.PARAMS['glen_a'] / 10
    elif glens_creep_parameter.value in ['large', 'groß']:
        glen_a = cfg.PARAMS['glen_a'] * 10

### set fs

In [ ]:
def set_fs():
    global fs

    if sliding_parameter.value in ['no sliding', 'kein Gleiten']:
        fs = 0
    elif sliding_parameter.value in ['sliding', 'Gleiten']:
        fs = 5.7e-20

### set mb model

In [ ]:
def set_mb_model():
    global mb_model

    ELA = int(ELA_height.value)
    gradient = int(mb_gradient.value)

    mb_model = LinearMassBalance(ELA, grad=gradient)

### set glacier wide mb area

In [ ]:
def set_glacier_wide_mb_area():
    global glacier_wide_mb_area

    glacier_wide_mb_area = map_dx * (widths[:-1] + widths[1:]) / 2

### set glacier wide mb heights

In [ ]:
def set_glacier_wide_mb_heights():
    global glacier_wide_mb_heights

    glacier_wide_mb_heights = (bed_h[:-1] + bed_h[1:]) / 2

### set glacier wide mb height mb

In [ ]:
def set_glacier_wide_mb_height_mb():
    global glacier_wide_mb_height_mb

    glacier_wide_mb_height_mb = mb_model.get_annual_mb(
        glacier_wide_mb_heights) * cfg.SEC_IN_YEAR

### set glacier wide mb

In [ ]:
def set_glacier_wide_mb():
    global glacier_wide_mb

    glacier_wide_mb = (np.sum((glacier_wide_mb_area *
                               glacier_wide_mb_height_mb)[model.fls[-1].thick[:-1] > 0]) /
                       np.sum(glacier_wide_mb_area[model.fls[-1].thick[:-1] > 0]))

## Define functions to initialize a model and plots

Functions initialize a new model (set all parameters and define new `FlowlineModel`) and also set all plots to there default appearance

### init model

In [ ]:
def init_model():
    global model

    # set all parameters for the model according to the choosen parameters from the menu
    set_mb_model()
    set_bed_h()
    set_widths()
    set_glen_a()
    set_fs()

    # set values for calculating the glacier wide mass balance
    set_glacier_wide_mb_area()
    set_glacier_wide_mb_heights()
    set_glacier_wide_mb_height_mb()

    # set glacier surface height to bed height because at the beginning there is no glacier
    surface_h = bed_h

    # initialize flowline with rectangular shape of glacier
    model_flowline = RectangularBedFlowline(surface_h=surface_h, bed_h=bed_h,
                                            widths=widths/map_dx, map_dx=map_dx)

    # initialize the actual model
    model = FlowlineModel(model_flowline, mb_model=mb_model, y0=0., glen_a=glen_a,
                          fs=fs, check_for_boundaries=False)

    # initialize all plots with new model
    init_geometry_plot()
    init_timeseries_plot()

### init geometry plot

In [ ]:
def init_geometry_plot():
    global run_surface_heights
    global glacier_outlines
    global geometry_figure_change_index

    # set surface height for plotting, at beginn old and new one are the same
    run_surface_heights = np.repeat(
        np.array(model.fls[-1].surface_h, ndmin=2), 2, axis=0)

    # set glacier outlines for plotting, at beginn old and new one are the same
    glacier_outlines = np.repeat(np.array(
        np.where(model.fls[-1].thick > 0, widths, nan_array), ndmin=2), 2, axis=0)

    # set all subplots for the geometry plot
    if show_geometry_plot.value:
        set_info_text(stage='new model')
    else:
        set_info_text(stage='switched off')
    set_bed_rock_curves()
    set_glacier_height_curve()
    set_mb_curve()
    set_width_curve()

    # force to refresh the geometry plot, the try statement is needed for the first initialization,
    # when the dynamic map do not yet exist
    try:
        geometry_figure.event(change=geometry_figure_change_index)
        geometry_figure_change_index = not geometry_figure_change_index
    except NameError:
        pass

### init timeseries plot

In [ ]:
def init_timeseries_plot():
    global glacier_volume
    global glacier_area
    global glacier_length
    global glacier_time
    global timeseries_vlin_pos
    global years_table
    global ELA_table
    global mb_gradient_table
    global glen_a_table
    global fs_table
    global volume_table
    global length_table
    global area_table
    global timeseries_table_change_index
    global timeseries_figure_change_index

    # set parameters for volume and length plot back to default
    glacier_volume = [0]
    glacier_area = [0]
    glacier_length = [0]
    glacier_time = [0]
    timeseries_vlin_pos = [0]

    # change the volume and length subplots
    set_glacier_volume_curve()
    set_glacier_area_curve()
    set_glacier_length_curve()

    # empty table entries back
    years_table = []
    ELA_table = []
    mb_gradient_table = []
    glen_a_table = []
    fs_table = []
    volume_table = []
    length_table = []
    area_table = []

    # set the new values in the table
    set_timeseries_table()

    # force the timeseries table to refresh, the try statement is needed for the first initialization,
    # when the dynamic map do not yet exist
    try:
        timeseries_table_DynamicMap.event(change=timeseries_table_change_index)
        timeseries_table_change_index = not timeseries_table_change_index
    except NameError:
        pass

    # force the timeseries figure to refresh, the try statement is needed for the first initialization,
    # when the dynamic map do not yet exist
    try:
        timeseries_figure.event(change=timeseries_figure_change_index)
        timeseries_figure_change_index = not timeseries_figure_change_index
    except NameError:
        pass

## Define function for creating curves

Functions to determine the look of each subplot or subplot element. After the definition the Functions are also called so all subplots are set to the default looking.

### set bed rock curves

In [ ]:
def set_bed_rock_curves():
    global bed_rock_height_curve
    global bed_rock_width_curve

    # create curve of bed rock for view from beside (top left subfigure, geometry plot)
    x_label = {'en': 'distance along glacier (km)',
               'de': 'Distanz entlang des Gletschers (km)'
               }[language]
    y_label = {'en': 'altitude (m)',
               'de': 'Höhe (m)'
               }[language]
    bed_rock_height_curve = hv.Area((distance_along_glacier, bed_h),
                                    x_label, y_label,
                                    ).opts(default_tools=default_tools_geometry,
                                           color='lightgray',
                                           line_alpha=0)

    # create curve of bed rock for view from above (bottom left subfigure, geometry plot)
    x_label = {'en': 'distance along glacier (km)',
               'de': 'Distanz entlang des Gletschers (km)'
               }[language]
    y_label = {'en': 'width (m)',
               'de': 'Weite (m)'
               }[language]
    bed_rock_width_curve = hv.Area((distance_along_glacier, -widths/2, widths/2),
                                   kdims=[x_label],
                                   vdims=[y_label, 'y2']
                                   # label='glacier rock bed'
                                   ).opts(default_tools=default_tools_geometry,
                                          color='lightgray',
                                          line_alpha=0)

### set glacier height curve

In [ ]:
def set_glacier_height_curve():
    global glacier_height_curve

    ELA = int(ELA_height.value)

    # set labels according to language
    x_label = {'en': 'distance along glacier (km)',
               'de': 'Distanz entlang des Gletschers (km)'
               }[language]
    y_label = {'en': 'altitude (m)',
               'de': 'Höhe (m)'
               }[language]
    glacier_label = {'en': 'glacier',
                     'de': 'Gletscher'
                     }[language]

    if not np.array_equal(run_surface_heights[0], bed_h):
        old_glacier = (distance_along_glacier, run_surface_heights[0])
    else:
        old_glacier = ([], [])

    glacier_height_curve = (hv.Area(old_glacier,
                                    x_label, y_label,
                                    # label='last'
                                    ).opts(default_tools=default_tools_geometry,
                                           line_dash='dashed',
                                           line_color='darkgray',
                                           color='white',
                                           alpha=0.6,
                                           line_alpha=1,
                                           line_width=3) *
                            hv.Area((distance_along_glacier, run_surface_heights[1]),
                                        x_label, y_label,
                                        label=glacier_label
                                    ).opts(default_tools=default_tools_geometry,
                                           color='white',
                                           line_color='blue',
                                           alpha=0.6,
                                           line_alpha=1,
                                           line_width=3))

    # looking for potential segments which can be colored if wanted
    index_of_segments = np.arange(np.sum(model.fls[-1].thick > 0))

    # show ice velocity in colors if wanted and if there is something to show
    if show_velocity.value and index_of_segments.size != 0:
        # calculate the mean velocity between two segments
        plot_velocity = [(np.array(model.get_diagnostics()['ice_velocity'])[i] +
                          np.array(model.get_diagnostics()['ice_velocity'])[i + 1]) / 2
                         for i in index_of_segments]

        # try statement is necessery because there are some problems with 'linear_cmap'
        # when the app is started in the notebook, and it works when the app is shown
        # in the browser seperatly
        try:
            # is necessary so the line_color is the same as the filled color of the polygons
            velocity_mapper = []
            velocity_mapper = linear_cmap(field_name='velocity', palette=colormap,
                                          low=np.min(plot_velocity),
                                          high=np.max(plot_velocity))

            # actual colored velocity
            glacier_height_curve *= hv.Polygons([{'x': [distance_along_glacier[i],
                                                        distance_along_glacier[i + 1],
                                                        distance_along_glacier[i + 1],
                                                        distance_along_glacier[i]],
                                                  'y': [bed_h[i],  # 0,
                                                        bed_h[i+1],  # 0,
                                                        run_surface_heights[1][i + 1],
                                                        run_surface_heights[1][i]],
                                                  'velocity': plot_velocity[i]}
                                                 for i in index_of_segments],
                                                vdims='velocity').opts(default_tools=default_tools_geometry,
                                                                       line_color=velocity_mapper,
                                                                       cmap=colormap)

        except:
            glacier_height_curve *= hv.Polygons([{'x': [distance_along_glacier[i],
                                                        distance_along_glacier[i + 1],
                                                        distance_along_glacier[i + 1],
                                                        distance_along_glacier[i]],
                                                  'y': [bed_h[i],  # 0,
                                                        bed_h[i+1],  # 0,
                                                        run_surface_heights[1][i + 1],
                                                        run_surface_heights[1][i]],
                                                  'velocity': plot_velocity[i]}
                                                 for i in index_of_segments],
                                                vdims='velocity').opts(default_tools=default_tools_geometry,
                                                                       line_color=None,
                                                                       cmap=colormap)

    else:
        # when there is no glacier or colors should not be shown
        # this is needed so bokeh knows about the Polygons element
        glacier_height_curve *= hv.Polygons([{'x': [],
                                              'y': [],
                                              'velocity': [0]}],
                                            vdims='velocity').opts(default_tools=default_tools_geometry)

    # add ELA to plot
    glacier_height_curve *= hv.Curve((distance_along_glacier, np.repeat(ELA, np.size(distance_along_glacier))),
                                     x_label, y_label,
                                     ).opts(default_tools=default_tools_geometry,
                                            line_dash='dashed',
                                            line_width=3,
                                            color='black')

### set mb curve

In [ ]:
def set_mb_curve():
    global mb_curve

    ELA = int(ELA_height.value)

    # set labels according to language
    x_label = {'en': 'annual mass balance (mm w.e./year)',
               'de': 'jährliche Massenbilanz (mm w.e/Jahr)'
               }[language]
    y_label = {'en': 'altitude (m)',
               'de': 'Höhe (m)'
               }[language]
    mass_gain_label = {'en': 'mass gain',
                       'de': 'Massengewinn'
                       }[language]
    mass_loss_label = {'en': 'mass loss',
                       'de': 'Massenverlust'
                       }[language]
    mass_balance_label = {'en': 'mass balance',
                          'de': 'Massenbilanz'
                          }[language]

    # define colors for mass gain/loss
    mass_gain_color = 'green'
    mass_loss_color = 'red'

    # define the range of the mass balance curve wich should be shown
    x_range_of_mb_curve = np.array([-20, 20])
    y_range_of_mb_curve = np.array([0, 4500])

    # get the annual mass balance for the given range of the x axis
    annual_mb = mb_model.get_annual_mb(y_range_of_mb_curve) * cfg.SEC_IN_YEAR

    # get the index for the current glacier extent
    glacier_extent_index = np.sum(run_surface_heights[1]-bed_h > 0)

    if glacier_extent_index == 0:
        mass_gain = ([], [], [])
        mass_loss = ([], [], [])

    elif run_surface_heights[1][glacier_extent_index] >= ELA:
        mass_gain = ([x_range_of_mb_curve],
                     [np.repeat(run_surface_heights[1]
                                [glacier_extent_index], 2)],
                     [np.repeat(run_surface_heights[1][0], 2)])
        mass_loss = ([], [], [])

    else:
        mass_gain = ([x_range_of_mb_curve],
                     [np.repeat(ELA, 2)],
                     [np.repeat(run_surface_heights[1][0], 2)])
        mass_loss = ([x_range_of_mb_curve],
                     [np.repeat(ELA, 2)],
                     [np.repeat(run_surface_heights[1]
                                [glacier_extent_index], 2)])

    mb_curve = (hv.Area(mass_gain,
                        kdims=[x_label],
                        # vdims have to be different
                        vdims=[y_label, 'y2'],
                        label=mass_gain_label
                        ).opts(default_tools=default_tools_geometry,
                               color=mass_gain_color,
                               line_alpha=0) *
                hv.Area(mass_loss,
                            kdims=[x_label],
                            # vdims have to be different
                            vdims=[y_label, 'y2'],
                            label=mass_loss_label
                        ).opts(default_tools=default_tools_geometry,
                               color=mass_loss_color,
                               line_alpha=0) *
                hv.Curve((annual_mb, y_range_of_mb_curve),
                         x_label,
                         y_label,
                         label=mass_balance_label
                         ).opts(default_tools=default_tools_geometry,
                                color='black') *
                hv.Curve((x_range_of_mb_curve, np.repeat(ELA, 2)),
                         x_label,
                         y_label,
                         label='ELA'
                         ).opts(default_tools=default_tools_geometry,
                                line_dash='dashed',
                                line_width=3,
                                color='black'))

### set width curve

In [ ]:
def set_ELA_x_position():
    global ELA_x_position

    # find index for ELA position on x-axis (distance along glacier)
    ELA_x_position = (np.abs(bed_h - ELA_height.value)).argmin()

In [ ]:
def set_width_curve():
    global width_curve

    # set the ELA position on the x axis
    set_ELA_x_position()

    # set labels according to language
    x_label = {'en': 'distance along glacier (km)',
               'de': 'Distanz entlang des Gletschers (km)'
               }[language]
    y_label = {'en': 'width (m)',
               'de': 'Weite (m)'
               }[language]

    # define lower and upper y limit which is shown
    y_min = -300  # in m
    y_max = 300

    if not np.array_equal(run_surface_heights[0], bed_h):
        old_glacier = (distance_along_glacier, -
                       glacier_outlines[0]/2, glacier_outlines[0]/2)
    else:
        old_glacier = ([], [], [])

    width_curve = (hv.Area(old_glacier,
                           kdims=[x_label],
                           vdims=[y_label, 'y2']
                           # label='old'
                           ).opts(default_tools=default_tools_geometry,
                                  line_dash='dashed',
                                  color='white',
                                  line_color='darkgray',
                                  alpha=0.6,
                                  line_alpha=1,
                                  line_width=3) *
                   hv.Area((distance_along_glacier, -glacier_outlines[1]/2, glacier_outlines[1]/2),
                               kdims=[x_label],
                               vdims=[y_label, 'y2']
                               # label='new'
                           ).opts(default_tools=default_tools_geometry,
                                  color='white',
                                  line_color='blue',
                                  alpha=0.6,
                                  line_alpha=1,
                                  line_width=3))
    # looking for potential segments which can be colored if wanted
    index_of_segments = np.arange(np.sum(model.fls[-1].thick > 0))

    # show ice velocity in colors if wanted and there is something to show
    if show_thickness.value and index_of_segments.size != 0:
        # calculate the mean thickness between two segments
        plot_thickness = [(np.array(model.get_diagnostics()['ice_thick'])[i] +
                           np.array(model.get_diagnostics()['ice_thick'])[i + 1]) / 2
                          for i in index_of_segments]

        # try statement is necessery because there are some problems with 'linear_cmap'
        # when the app is started in the notebook, and it works when the app is shown
        # in the browser seperatly
        try:
            # is necessary so the line_color is the same as the filled color of the polygons
            thickness_mapper = linear_cmap(field_name='thickness', palette=colormap,
                                           low=np.min(plot_thickness),
                                           high=np.max(plot_thickness))

            # actual colored thickness
            width_curve *= hv.Polygons([{'x': [distance_along_glacier[i],
                                               distance_along_glacier[i + 1],
                                               distance_along_glacier[i + 1],
                                               distance_along_glacier[i]],
                                         'y': [-glacier_outlines[1][i]/2,
                                               -glacier_outlines[1][i + 1]/2,
                                               glacier_outlines[1][i + 1]/2,
                                               glacier_outlines[1][i]/2],
                                         'thickness': plot_thickness[i]}
                                        for i in index_of_segments],
                                       vdims='thickness').opts(default_tools=default_tools_geometry,
                                                               line_color=thickness_mapper,
                                                               cmap=colormap)
        except:
            width_curve *= hv.Polygons([{'x': [distance_along_glacier[i],
                                               distance_along_glacier[i + 1],
                                               distance_along_glacier[i + 1],
                                               distance_along_glacier[i]],
                                         'y': [-glacier_outlines[1][i]/2,
                                               -glacier_outlines[1][i + 1]/2,
                                               glacier_outlines[1][i + 1]/2,
                                               glacier_outlines[1][i]/2],
                                         'thickness': plot_thickness[i]}
                                        for i in index_of_segments],
                                       vdims='thickness').opts(default_tools=default_tools_geometry,
                                                               line_color=None,
                                                               cmap=colormap)
    else:
        # when there is no glacier or colors should not be shown
        # this is needed so bokeh knows about the Polygons element
        width_curve *= hv.Polygons([{'x': [],
                                     'y': [],
                                     'thickness': [0]}],
                                   vdims='thickness').opts(default_tools=default_tools_geometry)

    width_curve *= (hv.Curve((distance_along_glacier, np.repeat(0, np.size(distance_along_glacier))),
                             x_label,
                             y_label,
                             # label='center Flowline'
                             ).opts(default_tools=default_tools_geometry,
                                    color='black',
                                    line_width=1) *
                    hv.Curve((np.repeat(distance_along_glacier[ELA_x_position], 2), [y_min, y_max]),
                             x_label,
                             y_label,
                             # label='ELA'
                             ).opts(default_tools=default_tools_geometry,
                                    line_dash='dashed',
                                    line_width=3,
                                    color='black'))

    width_curve.opts(ylim=(y_min, y_max))

### set info text

In [ ]:
def set_info_text(stage='new model'):
    global info_text
    global last_info_text_stage

    # is needed when language is switched
    last_info_text_stage = stage

    text_color = 'black'

    if stage == 'new model':
        text = {'en': 'New Model geometry\ninitialised',
                'de': 'Neue Model Geometry\ninitialisiert'
                }[language]
    elif stage == 'switched off':
        text = {'en': 'This plot is currently not used/updated.\nTo switch it on go to "choose plots".',
                'de': 'Dieses Diagramm ist aktuell nicht aktiv\nAktivieren unter "Darstellung wählen"'
                }[language]
        text_color = 'red'
    elif stage == 'switched on':
        text = {'en': 'Plot is switched on again\nand will be updated with the next run!',
                'de': 'Diagramm ist aktiv und\n wird mit nächstem Modellauf aktualisiert'
                }[language]
        text_color = 'darkgreen'
    else:
        if stage == 'running model':
            text = {'en': 'Model is running:',
                    'de': 'Model läuft:'
                    }[language]
        elif stage == 'equilibrium':
            text = {'en': 'Model reached equilibrium',
                    'de': 'Model beendet und im Gleichgewicht'
                    }[language]
        elif stage == 'stopped model':
            text = {'en': 'Model finished',
                    'de': 'Model beendet'
                    }[language]
        elif stage == 'aborted model':
            text = {'en': 'Model aborted (max calculation time reached, see "model options")',
                    'de': 'Modellauf abgebrochen (maximale Zeit erreicht, siehe "Modeloptionen")'
                    }[language]

        # set (calculate) glacier wide mass balance
        set_glacier_wide_mb()
        text += {'en': ('\nTime: {:4.0f} years     Length: {:4.2f} km      Area: {:.2f} km²     Volume: {:.2f} km³\n'
                        'Max ice thickness: {:.0f} m          Max ice velocity: {:.0f} m/year\n'
                        'Glacier-wide mass-balance: {:.2} mm w.e./year'),
                 'de': ('\nZeit: {:4.0f} years     Länge: {:4.2f} km      Fläche: {:.2f} km²     Volumen: {:.2f} km³\n'
                        'Max Eisdicke: {:.0f} m          Max Eisgeschwindigkeit: {:.0f} m/year\n'
                        'Gletscherweite Massenbilanz: {:.2} mm w.e./year')
                 }[language]
        text = text.format(model.yr,
                           model.length_m / 1000,
                           model.area_km2,
                           model.volume_km3,
                           np.max(model.get_diagnostics()['ice_thick']),
                           np.max(model.get_diagnostics()[
                                  'ice_velocity']) * cfg.SEC_IN_YEAR,
                           glacier_wide_mb)

    info_text = hv.Text(0.5, 0.5, text).opts(default_tools=['save', 'reset'],
                                             show_frame=False,
                                             text_baseline='middle',
                                             text_align='center',
                                             text_font_size='9pt',
                                             text_font='courier',
                                             text_color=text_color,
                                             xaxis=None,
                                             yaxis=None)

### set glacier volume curve

In [ ]:
def set_glacier_volume_curve():
    global glacier_volume_curve

    # set labels according to language
    x_label = {'en': 'Time (years)',
               'de': 'Zeit (Jahre)'
               }[language]
    y_label = {'en': 'volume (km³)',
               'de': 'Volumen (km³)'
               }[language]

    glacier_volume_curve = hv.Curve((glacier_time, glacier_volume),
                                    x_label,
                                    y_label
                                    ).opts(default_tools=default_tools_timeseries,
                                           active_tools=default_active_tools_timeseries)

    # draws 11 helplines to seperate the sections
    for i in np.arange(11):
        # looks if the i'th line is needed, else the line will be drawn at zero
        try:
            vlin_pos = timeseries_vlin_pos[i]
        except IndexError:
            vlin_pos = 0

        # actual add the vertical lines
        glacier_volume_curve *= hv.VLine(vlin_pos).opts(default_tools=default_tools_timeseries,
                                                        active_tools=default_active_tools_timeseries,
                                                        color='black')

    # set default range to (0, 1)
    if glacier_volume == [0]:
        glacier_volume_curve.opts(xlim=(0, 1),
                                  ylim=(0, 1))
    else:
        glacier_volume_curve.opts(xlim=(np.nan, np.nan),
                                  ylim=(np.nan, np.nan))

    # add grid to plot
    grid_style = {'xgrid_line_color': 'white', 'ygrid_line_color': None,
                  'grid_line_width': 2}
    glacier_volume_curve.opts(show_grid=True,
                              gridstyle=grid_style,
                              bgcolor='#ebebeb')

    # is needed for dynamical axes
    glacier_volume_curve.opts(opts.Curve(framewise=True))

### set glacier area curve

In [ ]:
def set_glacier_area_curve():
    global glacier_area_curve

    # set labels according to language
    x_label = {'en': 'Time (years)',
               'de': 'Zeit (Jahre)'
               }[language]
    y_label = {'en': 'area (km²)',
               'de': 'Fläche (km²)'
               }[language]

    glacier_area_curve = hv.Curve((glacier_time, glacier_area),
                                  x_label,
                                  y_label
                                  ).opts(default_tools=default_tools_timeseries,
                                         active_tools=default_active_tools_timeseries)

    # draws 11 helplines to seperate the sections
    for i in np.arange(11):
        # looks if the i'th line is needed, else the line will be drawn at zero
        try:
            vlin_pos = timeseries_vlin_pos[i]
        except IndexError:
            vlin_pos = 0

        # actual add the vertical lines
        glacier_area_curve *= hv.VLine(vlin_pos).opts(default_tools=default_tools_timeseries,
                                                      active_tools=default_active_tools_timeseries,
                                                      color='black')

    # set default range to (0, 1)
    if glacier_area == [0]:
        glacier_area_curve.opts(xlim=(0, 1),
                                ylim=(0, 1))
    else:
        glacier_area_curve.opts(xlim=(np.nan, np.nan),
                                ylim=(np.nan, np.nan))

    # add grid to plot
    grid_style = {'xgrid_line_color': 'white', 'ygrid_line_color': None,
                  'grid_line_width': 2}
    glacier_area_curve.opts(show_grid=True,
                            gridstyle=grid_style,
                            bgcolor='#ebebeb')

    # is needed for dynamical axes
    glacier_area_curve.opts(opts.Curve(framewise=True))

### set glacier length curve

In [ ]:
def set_glacier_length_curve():
    global glacier_length_curve

    # set labels according to language
    x_label = {'en': 'Time (years)',
               'de': 'Zeit (Jahre)'
               }[language]
    y_label = {'en': 'length (km)',
               'de': 'Länge (km)'
               }[language]

    glacier_length_curve = hv.Curve((glacier_time, glacier_length),
                                    x_label,
                                    y_label
                                    ).opts(default_tools=default_tools_timeseries,
                                           active_tools=default_active_tools_timeseries)

    # variable to count how many 'empty' sections
    empty_sections_count = 0

    # draws 11 helplines to seperate the sections and add the section number
    for i in np.arange(11):
        # set the section number to default (if not needed)
        section_number_text = ' '
        section_number_x = 0
        section_number_y = 0

        # looks if the i'th line and text is needed
        try:
            vlin_pos = timeseries_vlin_pos[i]

            # looks if vertical line position is not zero and
            # then set the section number text and position
            if vlin_pos != 0:
                section_number_text = ('# ' + str(i - 1))
                section_number_x = ((timeseries_vlin_pos[i] -
                                     timeseries_vlin_pos[i - 1]) / 2 +
                                    timeseries_vlin_pos[i - 1])
                section_number_y = np.max(glacier_length) / 4
        except IndexError:
            vlin_pos = 0

        if vlin_pos == 0:
            empty_sections_count += 1

        # actual add the vertical lines
        glacier_length_curve *= hv.VLine(vlin_pos).opts(default_tools=default_tools_timeseries,
                                                        active_tools=default_active_tools_timeseries,
                                                        color='black')

        # actual add the section number
        glacier_length_curve *= hv.Text(section_number_x,
                                        section_number_y,
                                        section_number_text
                                        ).opts(default_tools=default_tools_timeseries,
                                               active_tools=default_active_tools_timeseries)

    # looks if the plot is activated and if there is already someting to show,
    # if there is nothing to show the text tells that the plot is ready
    if show_timeseries_plot.value and len(glacier_length) == 1:
        glacier_length_curve *= hv.Text(0.5,
                                        0.5,
                                        {'en': 'plot is ready to use \nand update with the next run',
                                         'de': 'Diagramm bereit und\nwird mit nächstem Modellauf aktualisiert'
                                         }[language]
                                        ).opts(default_tools=default_tools_timeseries,
                                               active_tools=default_active_tools_timeseries,
                                               text_color='darkgreen')
    elif not show_timeseries_plot.value:
        glacier_length_curve *= hv.Text(0.5,
                                        0.5,
                                        {'en': 'plot is switched off \nto switch on look at "choose plots"',
                                         'de': 'Diagramm nicht aktiv\nAktivieren unter "Diagramm wählen"'
                                         }[language]
                                        ).opts(default_tools=default_tools_timeseries,
                                               active_tools=default_active_tools_timeseries,
                                               text_color='red'
                                               )
    else:
        glacier_length_curve *= hv.Text(0,
                                        0,
                                        ' '
                                        ).opts(default_tools=default_tools_timeseries,
                                               active_tools=default_active_tools_timeseries)

    # set default range to (0, 1)
    if glacier_length == [0]:
        glacier_length_curve.opts(xlim=(0, 1),
                                  ylim=(0, 1))
    else:
        glacier_length_curve.opts(xlim=(np.nan, np.nan),
                                  ylim=(np.nan, np.nan))

    # add grid to plot
    grid_style = {'xgrid_line_color': 'white', 'ygrid_line_color': None,
                  'grid_line_width': 2}
    glacier_length_curve.opts(show_grid=True,
                              gridstyle=grid_style,
                              bgcolor='#ebebeb')

    # is needed for dynamical axes
    glacier_length_curve.opts(opts.Curve(framewise=True))

### set timeseries table

In [ ]:
def set_timeseries_table():
    global timeseries_table

    table_heading_1 = {'en': ['Years', 'ELA (m)', 'gradient (mm w.e./year/m)', 'Glen\'s creep', 'sliding'],
                       'de': ['Jahre', 'ELA (m)', 'Gradient (mm w.e./Jahr/m)', 'Glen\'s creep', 'Gleiten']
                       }[language]
    table_heading_2 = {'en': ['length (km)', 'area (km²)', 'volume (km³)'],
                       'de': ['Länge (km)', 'Fläche (km²)', 'Volumen (km³)']
                       }[language]

    sliding_table = np.where(fs_table,
                             {'en': 'Yes', 'de': 'Ja'}[language],
                             {'en': 'No', 'de': 'Nein'}[language])
    timeseries_table = hv.Table((years_table, ELA_table, mb_gradient_table,
                                 glen_a_table, sliding_table, length_table,
                                 area_table, volume_table),
                                table_heading_1,
                                table_heading_2
                                ).opts(width=540)

## Define function to run model

In [ ]:
def run_the_model(stage='beginner mode'):
    global model
    global run_surface_heights
    global glacier_outlines
    global timeseries_table_change_index
    global timeseries_figure_change_index
    global geometry_figure_change_index

    if stage == 'beginner mode':
        # initialize a new model
        init_model()

    elif stage == 'advance equilibrium' or stage == 'advance years':
        # initialize model with old extend and new parameters
        set_mb_model()
        set_glen_a()
        set_fs()
        model = FlowlineModel(model.fls[-1], mb_model=mb_model, y0=0.,
                              glen_a=glen_a, fs=fs, check_for_boundaries=False)

        # determine glacier state at starting point
        run_surface_heights = np.repeat(
            np.array(model.fls[-1].surface_h, ndmin=2), 2, axis=0)
        glacier_outlines = np.repeat(np.array(
            np.where(model.fls[-1].thick > 0, widths, nan_array), ndmin=2), 2, axis=0)

        # set height dependent mass balance for glacier wide mass balance
        set_glacier_wide_mb_height_mb()

    if stage == 'beginner mode' or stage == 'advance equilibrium':
        # mostly same function as from OGGM FlowlineModel.run_until_equilibrium
        was_close_zero = 0
        t_rate = 1

        # first expression is the threshold for the volume change from one to the next calculated step,
        # second is the checking if the current model year is smaller than the choosen one,
        # third checks if the volume of the glacier is not always 0 (maximum five times)
        while (t_rate > 0.001) and (model.yr < max_calc_years) and (was_close_zero < 5):
            v_bef = model.volume_m3
            model.run_until(model.yr + dyears)
            v_af = model.volume_m3
            if np.isclose(v_bef, 0., atol=1):
                t_rate = 1
                was_close_zero += 1
            else:
                t_rate = np.abs(v_af - v_bef) / v_bef

            # check if geometry plot is wanted
            if show_geometry_plot.value:
                # set text with current year, volume, thickness and velocity
                set_info_text(stage='running model')

                # save new surface/outline and show it on plot
                run_surface_heights[1] = np.array(
                    model.fls[-1].surface_h, ndmin=2)
                glacier_outlines[1] = np.array(
                    np.where(model.fls[-1].thick > 0, widths, nan_array), ndmin=2)
                set_glacier_height_curve()
                set_width_curve()
                set_mb_curve()
                geometry_figure.event(change=geometry_figure_change_index)
                geometry_figure_change_index = not geometry_figure_change_index

            # check if timeseries plot is wanted
            if show_timeseries_plot.value:
                # save volume, area, length and time of current state
                glacier_volume.append(model.volume_km3)
                glacier_area.append(model.area_km2)
                glacier_length.append(model.length_m/1000)
                glacier_time.append(glacier_time[-1] + dyears)

                # set volume, area and length curves with new values
                set_glacier_volume_curve()
                set_glacier_area_curve()
                set_glacier_length_curve()

                # force a refresh of the timeseries plot
                timeseries_figure.event(change=timeseries_figure_change_index)
                timeseries_figure_change_index = not timeseries_figure_change_index

        # check if geometry plot is wanted
        if show_geometry_plot.value:
            if model.yr >= max_calc_years:
                set_info_text(stage='aborted model')
            else:
                # set text with equilibrium year
                set_info_text(stage='equilibrium')

            geometry_figure.event(change=geometry_figure_change_index)
            geometry_figure_change_index = not geometry_figure_change_index

    elif stage == 'advance years':
        while (model.yr < run_model_years.value):
            model.run_until(model.yr + dyears)

            # check if geometry plot is wanted
            if show_geometry_plot.value:
                # set text with current year and volume
                set_info_text(stage='running model')

                # save new surface/outline and show it on plot
                run_surface_heights[1] = np.array(
                    model.fls[-1].surface_h, ndmin=2)
                glacier_outlines[1] = np.array(
                    np.where(model.fls[-1].thick > 0, widths, nan_array), ndmin=2)
                set_glacier_height_curve()
                set_width_curve()
                set_mb_curve()
                geometry_figure.event(change=geometry_figure_change_index)
                geometry_figure_change_index = not geometry_figure_change_index

            # check if timeseries plot is wanted
            if show_timeseries_plot.value:
                # save volume, area, length and time of current state
                glacier_volume.append(model.volume_km3)
                glacier_area.append(model.area_km2)
                glacier_length.append(model.length_m/1000)
                glacier_time.append(glacier_time[-1] + dyears)

                # set volume, area and length curves with new values
                set_glacier_volume_curve()
                set_glacier_area_curve()
                set_glacier_length_curve()

                # force a refresh of the timeseries plot
                timeseries_figure.event(change=timeseries_figure_change_index)
                timeseries_figure_change_index = not timeseries_figure_change_index

        # check if geometry plot is wanted
        if show_geometry_plot.value:
            # set text after calculation
            set_info_text(stage='stopped model')
            geometry_figure.event(change=geometry_figure_change_index)
            geometry_figure_change_index = not geometry_figure_change_index

    # check if timeseries plot is wanted
    if show_timeseries_plot.value:
        # store time for vertical lines in volume and length plot
        timeseries_vlin_pos.append(glacier_time[-1])

        # set volume, area and length curves with new vertical line
        set_glacier_volume_curve()
        set_glacier_area_curve()
        set_glacier_length_curve()
        timeseries_figure.event(change=timeseries_figure_change_index)
        timeseries_figure_change_index = not timeseries_figure_change_index

        # store values for table
        years_table.append(int(model.yr))
        ELA_table.append(ELA_height.value)
        mb_gradient_table.append(mb_gradient.value)
        glen_a_table.append(glens_creep_parameter.value)
        if sliding_parameter.value == 'sliding':
            fs_table.append(True)
        else:
            fs_table.append(False)
        volume_table.append(np.round(model.volume_km3, decimals=2))
        length_table.append(np.round(model.length_m/1000, decimals=2))
        area_table.append(np.round(model.area_km2, decimals=2))

        # set the new values in the table
        set_timeseries_table()

        # force the table to refresh
        timeseries_table_DynamicMap.event(change=timeseries_table_change_index)
        timeseries_table_change_index = not timeseries_table_change_index

## Functions for dynamic maps (arrangement of subplots)

This functions define how the subplots will be arranged.

### change geometry figure

In [ ]:
def change_geometry_figure(change=True):
    return ((glacier_height_curve * bed_rock_height_curve).opts(legend_position='bottom_left',
                                                                legend_cols=2,
                                                                xaxis='top',
                                                                bgcolor='lightblue',
                                                                responsive=True,
                                                                aspect=2.5) +
            (mb_curve).opts(legend_position='bottom_right',
                            legend_cols=4,
                            xaxis='top',
                            yaxis='right',
                            bgcolor='lightgray',
                            responsive=True,
                            aspect=2.5) +
            (bed_rock_width_curve * width_curve).opts(responsive=True,
                                                      aspect=6.1,
                                                      yformatter='%4.0f',
                                                      bgcolor='darkgray',
                                                      labelled=['y']) +
            (info_text).opts(responsive=True,
                             aspect=6.1)
            ).opts(sizing_mode='stretch_both',
                   toolbar='right').cols(2)

### change timeseries figure

In [ ]:
def change_timeseries_figure(change=True):
    return ((glacier_volume_curve).opts(responsive=True,
                                        xaxis='top',
                                        aspect=4.3,
                                        max_height=120) +
            (glacier_area_curve).opts(responsive=True,
                                      aspect=6.4,
                                      labelled=['y'],
                                      xaxis=None,
                                      max_height=90) +
            (glacier_length_curve).opts(responsive=True,
                                        labelled=['y'],
                                        aspect=5.3,
                                        max_height=90)
            ).opts(sizing_mode='stretch_both',
                   toolbar='left').cols(1)

### change timeseries table

In [ ]:
def change_timeseries_table(change=True):
    return timeseries_table.opts(height=300)

## streamers and dynamic maps

before creating the dynamic maps initialise the model and all subplots with the default values

In [ ]:
init_model()

dynamic map and stream for geometry plot

In [ ]:
geometry_figure_stream = Stream.define('geometry_figure_stream', change=False)

geometry_figure = hv.DynamicMap(
    change_geometry_figure, streams=[geometry_figure_stream()])

dynamic maps and streams for timeseries plot (figure and table)

In [ ]:
timeseries_figure_stream = Stream.define(
    'timeseries_figure_stream', change=False)

timeseries_figure = hv.DynamicMap(
    change_timeseries_figure, streams=[timeseries_figure_stream()])

In [ ]:
timeseries_table_stream = Stream.define(
    'timeseries_table_stream', change=False)

timeseries_table_DynamicMap = hv.DynamicMap(
    change_timeseries_table, streams=[timeseries_table_stream()])

## put figures together in tab menu

In [ ]:
figures = pn.Tabs(('', []),
                  ('', []),
                  tabs_location='below',
                  sizing_mode='stretch_both')

figures_tab1 = geometry_figure
figures_tab2 = pn.Row(pn.Column(timeseries_figure,
                                max_width=600,
                                max_height=300,
                                sizing_mode='stretch_both'),
                      timeseries_table_DynamicMap,
                      sizing_mode='stretch_both')

# is necessary to change tabs title language


def set_figures_tabs():
    figures.__setitem__(0, ({'en': 'geometry plot',
                             'de': 'Geometrie Darstellung'
                             }[language],
                            figures_tab1))

    figures.__setitem__(1, ({'en': 'timeseries plot',
                             'de': 'Zeitreihen Darstellung'
                             }[language],
                            figures_tab2))


set_figures_tabs()

## App

In [ ]:
pn.Column(header, menu, figures, sizing_mode='stretch_both').servable()

As long as you are running this notebook "live" (in Jupyter, not viewing a website or a static copy), the above notebook cell should contain the fully operational dashboard here in the notebook. You can also launch the dashboard at a separate port that shows up in a new browser tab, either by changing .servable() to .show() above and re-executing that cell, or by leaving the cell as it is and running bokeh serve --show simulator.ipynb.